In [3]:
import os
# Find the latest version of spark 2.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.0'
spark_version = 'spark-3.0.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:6 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:8 http://security.ubuntu.com/ubuntu bionic-security InRelease
Hit:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:12 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:14 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Reading package lists... Done


In [4]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2021-02-10 22:38:51--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  5.96MB/s    in 0.2s    

2021-02-10 22:38:52 (5.96 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [5]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("FinalProject").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [6]:
from pyspark import SparkFiles

In [7]:
#import library psycopyg2
import psycopg2

#import library pandas
import pandas as pd

#import library sqlio
import pandas.io.sql as sqlio

#create database connection variable 
conn = psycopg2.connect(user="postgres", password="ilovegroup3", host="finalproject.c969jxlkzz2v.us-east-2.rds.amazonaws.com", database="housingdata")

/usr/local/lib/python3.6/dist-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


In [8]:
#define query
query = "select * from census_housing_data"

In [9]:
#execute query and save it to a variable
dataset = sqlio.read_sql_query(query,conn)

In [10]:
dataset

,zipcode,city,zhvi,total_pop,male,female,med_age,race_w,race_aa,race_nat,race_as,race_api,race_oth,eth_hisp,eth_nonhisp,median_income
0,61531.0,Farmington,84009.17,3335.0,1647.0,1688.0,40.9,3274.0,0.0,38.0,77.0,0.0,8.0,77.0,3258.0,56528.0
1,61532.0,Forest City,46023.33,645.0,322.0,323.0,36.7,644.0,0.0,0.0,0.0,0.0,1.0,0.0,645.0,51833.0
2,61533.0,Glasford,117554.17,2509.0,1274.0,1235.0,41.9,2489.0,0.0,0.0,0.0,0.0,20.0,27.0,2482.0,60250.0
3,61534.0,Green Valley,111049.92,1806.0,946.0,860.0,42.4,1796.0,9.0,3.0,0.0,0.0,0.0,25.0,1781.0,57917.0
4,61535.0,Groveland,180972.58,1697.0,966.0,731.0,41.6,1662.0,16.0,41.0,0.0,0.0,0.0,0.0,1697.0,81964.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1380,62313.0,Basco,NaN,378.0,192.0,186.0,51.5,377.0,1.0,2.0,0.0,0.0,0.0,0.0,378.0,NaN
1381,61059.0,Nora,NaN,212.0,87.0,125.0,31.4,203.0,0.0,9.0,0.0,0.0,0.0,1.0,211.0,62500.0
1382,62036.0,Golden Eagle,NaN,803.0,450.0,353.0,56.4,803.0,0.0,0.0,0.0,0.0,0.0,0.0,803.0,NaN
1383,62663.0,Manchester,NaN,259.0,131.0,128.0,49.3,259.0,4.0,1.0,0.0,0.0,0.0,0.0,259.0,NaN


In [11]:
income_by_city = dataset.groupby(["city"]).mean()["median_income"]
income_by_city_df = income_by_city.to_frame()
income_by_city_df

,median_income
city,
Abingdon,44570.0
Adair,68594.0
Addieville,NaN
Addison,64915.0
Albany,55000.0
...,...
Yale,NaN
Yates City,56607.0
Yorkville,95878.0


In [12]:
housingprice_by_city = dataset.groupby(["city"]).median()["zhvi"]
housingprice_by_city_df = housingprice_by_city.to_frame()
housingprice_by_city_df

,zhvi
city,
Abingdon,59676.67
Adair,62941.75
Addieville,127963.58
Addison,263669.50
Albany,117470.75
...,...
Yale,56292.50
Yates City,87263.75
Yorkville,253435.08


In [13]:
age_by_city = dataset.groupby(["city"]).mean()["med_age"]
age_by_city_df = age_by_city.to_frame()
age_by_city_df

,med_age
city,
Abingdon,39.4
Adair,54.6
Addieville,39.8
Addison,36.7
Albany,45.4
...,...
Yale,44.8
Yates City,45.2
Yorkville,35.5


In [14]:
demo_by_city = dataset.groupby(['city'])["total_pop", "male", "female", "race_w", "race_aa", "race_nat", "race_as", "race_api", "race_oth", "eth_hisp", "eth_nonhisp"].apply(lambda x : x.astype(int).sum())
demo_by_city

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  """Entry point for launching an IPython kernel.


,total_pop,male,female,race_w,race_aa,race_nat,race_as,race_api,race_oth,eth_hisp,eth_nonhisp
city,,,,,,,,,,,
Abingdon,3892,1731,2161,3865,55,62,0,0,0,39,3853
Adair,444,274,170,444,0,0,0,0,0,12,432
Addieville,1242,620,622,1232,0,10,36,3,17,18,1224
Addison,39216,19281,19935,29293,1812,401,3905,38,4986,15603,23613
Albany,1121,531,590,1119,0,2,0,0,2,47,1074
...,...,...,...,...,...,...,...,...,...,...,...
Yale,430,200,230,409,18,70,0,0,0,0,430
Yates City,1191,589,602,1186,32,20,0,0,0,0,1191
Yorkville,24886,12460,12426,21503,1883,120,849,0,1193,3049,21837


In [15]:
demo_by_city_df = demo_by_city.join(age_by_city_df)
demo_by_city_df

,total_pop,male,female,race_w,race_aa,race_nat,race_as,race_api,race_oth,eth_hisp,eth_nonhisp,med_age
city,,,,,,,,,,,,
Abingdon,3892,1731,2161,3865,55,62,0,0,0,39,3853,39.4
Adair,444,274,170,444,0,0,0,0,0,12,432,54.6
Addieville,1242,620,622,1232,0,10,36,3,17,18,1224,39.8
Addison,39216,19281,19935,29293,1812,401,3905,38,4986,15603,23613,36.7
Albany,1121,531,590,1119,0,2,0,0,2,47,1074,45.4
...,...,...,...,...,...,...,...,...,...,...,...,...
Yale,430,200,230,409,18,70,0,0,0,0,430,44.8
Yates City,1191,589,602,1186,32,20,0,0,0,0,1191,45.2
Yorkville,24886,12460,12426,21503,1883,120,849,0,1193,3049,21837,35.5


In [16]:
demo_by_city = demo_by_city_df.join(income_by_city_df)
demo_by_city

,total_pop,male,female,race_w,race_aa,race_nat,race_as,race_api,race_oth,eth_hisp,eth_nonhisp,med_age,median_income
city,,,,,,,,,,,,,
Abingdon,3892,1731,2161,3865,55,62,0,0,0,39,3853,39.4,44570.0
Adair,444,274,170,444,0,0,0,0,0,12,432,54.6,68594.0
Addieville,1242,620,622,1232,0,10,36,3,17,18,1224,39.8,NaN
Addison,39216,19281,19935,29293,1812,401,3905,38,4986,15603,23613,36.7,64915.0
Albany,1121,531,590,1119,0,2,0,0,2,47,1074,45.4,55000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
Yale,430,200,230,409,18,70,0,0,0,0,430,44.8,NaN
Yates City,1191,589,602,1186,32,20,0,0,0,0,1191,45.2,56607.0
Yorkville,24886,12460,12426,21503,1883,120,849,0,1193,3049,21837,35.5,95878.0


In [17]:
demo_by_city = demo_by_city.join(housingprice_by_city_df)
demo_by_city

,total_pop,male,female,race_w,race_aa,race_nat,race_as,race_api,race_oth,eth_hisp,eth_nonhisp,med_age,median_income,zhvi
city,,,,,,,,,,,,,,
Abingdon,3892,1731,2161,3865,55,62,0,0,0,39,3853,39.4,44570.0,59676.67
Adair,444,274,170,444,0,0,0,0,0,12,432,54.6,68594.0,62941.75
Addieville,1242,620,622,1232,0,10,36,3,17,18,1224,39.8,NaN,127963.58
Addison,39216,19281,19935,29293,1812,401,3905,38,4986,15603,23613,36.7,64915.0,263669.50
Albany,1121,531,590,1119,0,2,0,0,2,47,1074,45.4,55000.0,117470.75
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Yale,430,200,230,409,18,70,0,0,0,0,430,44.8,NaN,56292.50
Yates City,1191,589,602,1186,32,20,0,0,0,0,1191,45.2,56607.0,87263.75
Yorkville,24886,12460,12426,21503,1883,120,849,0,1193,3049,21837,35.5,95878.0,253435.08


In [18]:
demo_by_city.reset_index(inplace=True)
demo_by_city

,city,total_pop,male,female,race_w,race_aa,race_nat,race_as,race_api,race_oth,eth_hisp,eth_nonhisp,med_age,median_income,zhvi
0,Abingdon,3892,1731,2161,3865,55,62,0,0,0,39,3853,39.4,44570.0,59676.67
1,Adair,444,274,170,444,0,0,0,0,0,12,432,54.6,68594.0,62941.75
2,Addieville,1242,620,622,1232,0,10,36,3,17,18,1224,39.8,NaN,127963.58
3,Addison,39216,19281,19935,29293,1812,401,3905,38,4986,15603,23613,36.7,64915.0,263669.50
4,Albany,1121,531,590,1119,0,2,0,0,2,47,1074,45.4,55000.0,117470.75
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1241,Yale,430,200,230,409,18,70,0,0,0,0,430,44.8,NaN,56292.50
1242,Yates City,1191,589,602,1186,32,20,0,0,0,0,1191,45.2,56607.0,87263.75
1243,Yorkville,24886,12460,12426,21503,1883,120,849,0,1193,3049,21837,35.5,95878.0,253435.08
1244,Zeigler,1794,915,879,1786,6,29,7,0,29,34,1760,32.8,NaN,39465.17


In [20]:
demo_by_city = spark.createDataFrame(demo_by_city)
demo_by_city

DataFrame[city: string, total_pop: bigint, male: bigint, female: bigint, race_w: bigint, race_aa: bigint, race_nat: bigint, race_as: bigint, race_api: bigint, race_oth: bigint, eth_hisp: bigint, eth_nonhisp: bigint, med_age: double, median_income: double, zhvi: double]

In [ ]:
# Configure settings for RDS
mode = "append"
jdbc_url="jdbc:postgresql://finalproject.c969jxlkzz2v.us-east-2.rds.amazonaws.com:5432/housingdata"
config = {"user":"postgres",
          "password": "ilovegroup3",
          "driver":"org.postgresql.Driver"}

In [ ]:
# Write DataFrame to demo_by_city table in RDS
demo_by_city.write.jdbc(url=jdbc_url, table='demo_by_city', mode=mode, properties=config)